In [3]:
# Author: Arthur Prigent
# Email: aprigent@geomar.de

In [4]:
import xarray as xr 
#import xgcm 
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
import glob
import cartopy.crs as ccrs
import warnings
warnings.filterwarnings('ignore')
import matplotlib.ticker as mticker
import cartopy
from scipy import stats
import pandas as pd
from datetime import datetime
now = datetime.now()



def data_sub(data,lon_min,lon_max,lat_min,lat_max):
    
    '''Define a box between lon_min lon_max lat_min and lat_max and 
    extract the data in the box and drop everything else.
    
    
    Parameters
    ----------
    
    data : xarray_like
    Data to be subdomained. 
    
    lon_min : integer
    Longitude minimum of the subdomain
    
    lon_max : integer
    Longitude maximum of the subdomain
    
    lat_min : integer
    Latitude minimum of the subdomain
    
    lat_max : integer
    Latitude maximum of the subdomain
    
    Returns
    ---------
    
    data_sub : xarray_like
    Subdomain. 
    '''
    
    try:
        data_sub = data.where((  data.lon>=lon_min) & (data.lon<=lon_max) & (data.lat<=lat_max) & (data.lat>=lat_min),
                                                                          drop=True)
    except AttributeError:
        try:
            data_sub = data.where((  data.nav_lon>=lon_min) & (data.nav_lon<=lon_max) & (data.nav_lat<=lat_max) & (data.nav_lat>=lat_min),drop=True)
        except AttributeError:
            try:
                data_sub = data.where((  data.longitude>=lon_min) & (data.longitude<=lon_max) & (data.latitude<=lat_max) & (data.latitude>=lat_min),drop=True)
            except AttributeError:
                try:
                    data_sub = data.where((  data.x>=lon_min) & (data.x<=lon_max) & (data.y<=lat_max) &
                                      (data.y>=lat_min),drop=True)
                except AttributeError:
                    data_sub = data.where((  data.LON>=lon_min) & (data.LON<=lon_max) & (data.LAT<=lat_max) &
                                      (data.LAT>=lat_min),drop=True)
            
    

 
    
    return data_sub
import scipy.optimize as sc_o


def nandetrend(y):
    ''' Remove the linear trend from the data '''
    
    x = np.arange(0,y.shape[0],1)
    m, b, r_val, p_val, std_err = stats.linregress(x,np.array(y))
    y_detrended= np.array(y) - m*x -b
    return y_detrended



dir_cmip5 = '/data/user/aprigent/projects/uncertainty_ATL3_ABA/data/swift.dkrz.de/cmip5_data/'
dir_cmip6 = '/data/user/aprigent/projects/uncertainty_ATL3_ABA/data/swift.dkrz.de/cmip6_data/'



data_cmip_hist_tos = '/data/user/aprigent/projects/uncertainty_ATL3_ABA/data/swift.dkrz.de/cmip5_data/hist_rcp45/'
data_cmip_rcp_tos = '/data/user/aprigent/projects/uncertainty_ATL3_ABA/data/swift.dkrz.de/cmip5_data/rcp_85/'


path_fig='/data/user/aprigent/projects/uncertainty_ATL3_ABA/figures/'
path_data_out = '/data/user/aprigent/projects/uncertainty_ATL3_ABA/data/'

# List of models and scenario considered

In [5]:


model_list_CMIP5=['ACCESS1-0','ACCESS1-3','bcc-csm1-1-m','bcc-csm1-1','BNU-ESM',
            'CanESM2','CCSM4','CESM1-BGC','CESM1-CAM5','CMCC-CM','CMCC-CMS','CNRM-CM5','CSIRO-Mk3-6-0',
            'FGOALS-g2','GFDL-CM3','GFDL-ESM2G','GFDL-ESM2M','GISS-E2-H-CC','GISS-E2-H',
            'GISS-E2-R-CC','GISS-E2-R','HadGEM2-CC','HadGEM2-ES','inmcm4','IPSL-CM5A-LR','IPSL-CM5A-MR','IPSL-CM5B-LR',
            'MIROC-ESM-CHEM','MIROC-ESM','MIROC5','MPI-ESM-LR','MPI-ESM-MR','MRI-CGCM3','NorESM1-M','NorESM1-ME']

scenario_CMIP5=['hist_rcp45','rcp85']

len(model_list_CMIP5)

35

# Functions to get the ATL3 SST variability

In [6]:
def make_sst_atl3(sst_tmp):

    sst_tmp = sst_tmp.ts[:] - 273.15 # convert to degree C
    sst_tmp = xr.concat([sst_tmp[ :,:, 72:], sst_tmp[:, :, :72]], dim='lon')
    sst_tmp.coords['lon'] = (sst_tmp.coords['lon'] + 180) % 360 - 180
    sst_atl3 = data_sub(sst_tmp,-20,0,-3,3) # Take ATL3 box
    sst_atl3 = sst_atl3.mean(dim='lon').mean(dim='lat') # average of the ATL3 box

    return sst_atl3


def ano_norm_t(ds):
    
    '''Compute the anomalies by removing the monthly means. 
    The anomalies are normalized by their corresponding month.
    
    Parameters
    ----------
    
    ds : xarray_like
    Timeserie or 3d field.
    
    Returns
    -----------
    
    ano : xarray_like
    Returns the anomalies of var relative the climatology.
    
    ano_norm : xarray_like
    Returns the anomalies of var relative the climatology normalized by the standard deviation.
    
    '''    
    
    clim     = ds.groupby('time.month').mean('time')
    clim_std = ds.groupby('time.month').std('time')
    ano      = ds.groupby('time.month') - clim
    ano_norm = xr.apply_ufunc(lambda x, m, s: (x - m) / s,
                                    ds.groupby('time.month'),
                                    clim, clim_std)
    
    return ano, ano_norm


def compute_atl3_amplitude(sst_atl3,period_str,period_end):
    
    try:

        sst_atl3 = sst_atl3.sel(time=slice(datetime(period_str, 1, 1),
                                                         datetime(period_end, 12, 31)))
    except TypeError:
        sst_atl3['time'] = sst_atl3.indexes['time'].to_datetimeindex()
        sst_atl3 = sst_atl3.sel(time=slice(datetime(period_str, 1, 1),
                                                         datetime(period_end, 12, 31)))   
    
    xdata_1 = np.arange(0,sst_atl3.shape[0],1)
    ydata_1_tmp = np.array(sst_atl3)
    #print(sst_atl3.shape[0])
    
    sst_atl3 = sst_atl3.assign_coords(sst_dtd=('time',  nandetrend(ydata_1_tmp)))
    if sst_atl3.shape[0]>0:

        ssta_tmp,_ = ano_norm_t(sst_atl3.sst_dtd)
        amp_atl3_std = ssta_tmp.std(dim='time')
    else:
        amp_atl3_std = np.nan


    return amp_atl3_std

# Create the 31-year windows 

In [7]:
year_start= np.arange(1900,2070,1)
year_end= np.arange(1930,2100,1)


period_str_cmip5_hist = 1900
period_end_cmip5_hist = 2005

period_str_cmip5_rcp45 = 2006
period_end_cmip5_rcp45 = 2099

x = x(s,m,t) with x representing the Atlantic Nino amplitude

X is the low-pass filtered values of Atlantic Nino amplitude, X$_{f}$ is the long-term trend and $\epsilon$ the internal long-term variability

X(s,m,t) = X$_{f}$(s,m,t) + $\epsilon$(s,m,t) hence the dimension are s = 2, i.e. 2 scenarios, m =35, 35 models.

# Compute X(s,m,t)

In [8]:
year_end.shape[0]

170

In [9]:

## TOS ## 
amp_atl3_std = np.ones((len(scenario_CMIP5),len(model_list_CMIP5),year_start.shape[0]))*np.nan
for j in range(len(scenario_CMIP5)): ## loop on the scenarios
        print(scenario_CMIP5[j])
        for i in range(len(model_list_CMIP5)): ## loop on the models
            print(model_list_CMIP5[i])
            ## Take the historical data ##
            print('ts.hist_rcp45.'+model_list_CMIP5[i]+'.r1i1p1.r144x72.ncts.hist_rcp45.'+model_list_CMIP5[i]+'.r1i1p1.r144x72.nc')
            data_hist = xr.open_dataset(
                data_cmip_hist_tos+'ts.hist_rcp45.'+model_list_CMIP5[i]+'.r1i1p1.r144x72.ncts.hist_rcp45.'+model_list_CMIP5[i]+'.r1i1p1.r144x72.nc',
                decode_times=False)
            time = pd.date_range('1900-01-15', freq='M', periods=data_hist.ts.shape[0])
            data_hist['time'] = time
            data_hist_new = data_hist.sel(time=slice(datetime(period_str_cmip5_hist, 1, 1),
                                                         datetime(period_end_cmip5_hist, 12, 31)))
            
            ## Take the rcp45 data ##
            if scenario_CMIP5[j]=='hist_rcp45': 
                data_rcp = xr.open_dataset(
                data_cmip_hist_tos+'ts.'+scenario_CMIP5[j]+'.'+model_list_CMIP5[i]+'.r1i1p1.r144x72.ncts.'+scenario_CMIP5[j]+'.'+model_list_CMIP5[i]+'.r1i1p1.r144x72.nc',
                decode_times=False)
                time = pd.date_range('1900-01-15', freq='M', periods=data_rcp.ts.shape[0])
                data_rcp['time'] = time
                data_rcp_new = data_rcp.sel(time=slice(datetime(period_str_cmip5_rcp45, 1, 1),
                                                         datetime(period_end_cmip5_rcp45, 12, 31)))
               
            ## Take the rcp85 data ##
            elif scenario_CMIP5[j]=='rcp85':
                print('ts.rcp85.'+model_list_CMIP5[i]+'.r1i1p1.r144x72.ncts.rcp85.'+model_list_CMIP5[i]+'.r1i1p1.r144x72.nc')
                data_rcp = xr.open_dataset(
                data_cmip_rcp_tos+'ts.'+scenario_CMIP5[j]+'.'+model_list_CMIP5[i]+'.r1i1p1.r144x72.ncts.'+scenario_CMIP5[j]+'.'+model_list_CMIP5[i]+'.r1i1p1.r144x72.nc',
                decode_times=False)
                time = pd.date_range('2006-01-15', freq='M', periods=data_rcp.ts.shape[0])
                data_rcp['time'] = time
                data_rcp_new = data_rcp.sel(time=slice(datetime(period_str_cmip5_rcp45, 1, 1),
                                                         datetime(period_end_cmip5_rcp45, 12, 31)))
                
            ## Average over the ATL3 box ##
            sst_hist_atl3 = make_sst_atl3(data_hist_new)
            sst_rcp_atl3 = make_sst_atl3(data_rcp_new)
            
            ## concatenate the historical and the scenario runs ##
            sst_all_atl3 = xr.concat([sst_hist_atl3,sst_rcp_atl3],dim='time')
            
            ## Compute the 31- year running standard deviation of the ATL3-averaged SST anomalies ##
            print('start running mean')
            for k in range(170):
                #print(j)
                amp_atl3_std[j,i,k] = compute_atl3_amplitude(sst_all_atl3,year_start[k],year_end[k])
            print(' ')

hist_rcp45
ACCESS1-0
ts.hist_rcp45.ACCESS1-0.r1i1p1.r144x72.ncts.hist_rcp45.ACCESS1-0.r1i1p1.r144x72.nc
start running mean
 
ACCESS1-3
ts.hist_rcp45.ACCESS1-3.r1i1p1.r144x72.ncts.hist_rcp45.ACCESS1-3.r1i1p1.r144x72.nc
start running mean
 
bcc-csm1-1-m
ts.hist_rcp45.bcc-csm1-1-m.r1i1p1.r144x72.ncts.hist_rcp45.bcc-csm1-1-m.r1i1p1.r144x72.nc
start running mean


KeyboardInterrupt: 

# save the data into a netcdf file

In [19]:

scenario_CMIP5_new=['rcp45','rcp85']
time_new = pd.date_range('1915-06-01',freq='Y',periods=170)
running_mean_atl3_tmp = xr.Dataset({'amp_ATL3': (['scenario','model','time'], amp_atl3_std),
                               },
                      coords={'scenario': np.array(scenario_CMIP5_new),
                              'model': np.array(model_list_CMIP5),
                              'time':time_new} )
running_mean_atl3_tmp.to_netcdf(path_data_out+'ATL3_amplitude_CMIP5.nc')